<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 02 - Data Virtualization

In this tutorial you will learn how to create and query Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 


In [1]:
/* Create External File Format */

USE WideWorldImporters;
GO

IF NOT EXISTS(SELECT * FROM sys.external_file_formats WHERE name = 'csv_file')
BEGIN
    CREATE EXTERNAL FILE FORMAT csv_file
    WITH (
        FORMAT_TYPE = DELIMITEDTEXT,
        FORMAT_OPTIONS(
            FIELD_TERMINATOR = ',',
            STRING_DELIMITER = '0x22',
            FIRST_ROW = 2,
            USE_TYPE_DEFAULT = TRUE)
    );
END

Commands completed successfully.

Total execution time: 00:00:00.0788445

Commands completed successfully.

Total execution time: 00:00:00.8864665

In [2]:
/* Create External Data Source to the Storage Pool */
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlStoragePool')
BEGIN
    CREATE EXTERNAL DATA SOURCE SqlStoragePool
    WITH (LOCATION = 'sqlhdfs://service-master-pool:50070');
END

Commands completed successfully.

Total execution time: 00:00:00.3917940

In [3]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'partner_customers_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [partner_customers_hdfs]
    ("CustomerSource" VARCHAR(250) 
    , "CustomerName" VARCHAR(250) 
    , "EmailAddress" VARCHAR(250))
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/partner_customers',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.7815709

In [4]:
/* Read Data from HDFS using only T-SQL */

SELECT TOP 10 CustomerSource
, CustomerName
, EMailAddress
    FROM [partner_customers_hdfs] hdfs
WHERE EmailAddress LIKE '%wingtip%';
GO


(10 rows affected)

Total execution time: 00:00:06.1719983

CustomerSource,CustomerName,EMailAddress
AdventureWorks,Olga Alexandrova,olga@wingtiptoys.com
AdventureWorks,Bibi Javan,bibi@wingtiptoys.com
AdventureWorks,Maris Baltins,maris@wingtiptoys.com
AdventureWorks,Taneli Hahl,taneli@wingtiptoys.com
AdventureWorks,Parsa Reuser,parsa@wingtiptoys.com
AdventureWorks,Da-eun Kwak,da-eun@wingtiptoys.com
AdventureWorks,Duleep Srivastava,duleep@wingtiptoys.com
AdventureWorks,Kurt Balog,kurt@wingtiptoys.com
AdventureWorks,Miran Nikolic,miran@wingtiptoys.com
AdventureWorks,Indra Mudaliyar,indra@wingtiptoys.com


In [18]:
/* Now Join Those to show customers we currently have in a SQL Server Database 
and the Category they qre in the External Table */
USE WideWorldImporters;
GO

SELECT TOP 10 a.FullName
  , b.CustomerSource
  FROM Application.People a
  INNER JOIN partner_customers_hdfs b  ON a.FullName = b.CustomerName;
  GO

Commands completed successfully.

Total execution time: 00:00:00.0798501

(10 rows affected)

Total execution time: 00:00:08.6235983

FullName,CustomerSource
Deeptendu Das,AdventureWorks
Bhaagyalakshmi Rajanala,AdventureWorks
Rohan Das,AdventureWorks
Bhaanusree Ankitham,AdventureWorks
Aamdaal Kamasamudram,AdventureWorks
Karita Rosberg,AdventureWorks
Baebeesarasvati Kamasamudram,AdventureWorks
Hana Palatova,AdventureWorks
Marcello Dellucci,AdventureWorks
Shobha Raju,AdventureWorks


## Next Steps: Continue on to Working with the SQL Server Data Pool